In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from statannotations.Annotator import Annotator

from importlib import reload
reload(base)

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)

In [ ]:
output_path = rd.rootdir/'output'/'for-review'

### Varying number of target sites for miR-FF5

In [ ]:
# Setup data loading
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp45_1_path = base_path/'2022.07.15_exp45'
exp45_2_path = base_path/'2023.01.14_exp45.2'/'export'
exp45_3_path = base_path/'2023.02.10_exp45.3'/'export'

plates = pd.DataFrame({
    'data_path': [exp45_1_path/'293T'/'export', exp45_2_path, exp45_3_path,],
    'yaml_path': [exp45_1_path/'exp45_wells2.yaml', exp45_2_path/'exp45.2_wells2.yaml', exp45_2_path/'exp45.2_wells2.yaml'],
    'biorep': [1, 2, 3,],
    'exp': ['exp45', 'exp45.2', 'exp45.3',]
})
cache_path = output_path/'tuning-old.gzip'

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
# Load data
data = pd.DataFrame()
channel_list = ['mRuby2-A','mCherry-A','tagBFP-A']

if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)
    for c in channel_list: data = data[data[c]>0]
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx', style='designs')
metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'] = metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'].apply(base.get_light_color)
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mRuby2-A',]
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates['marker'] = 'tagBFP-A'
gates['output'] = 'mRuby2-A'

data['marker'] = data['tagBFP-A']
data['output'] = data['mRuby2-A']

display(gates)

In [ ]:
exp_palette = {'exp45': base.colors['teal'], 'exp45.2': base.colors['orange'], 'exp45.3': base.colors['red']}

In [ ]:
plot_df = data.groupby(['exp','construct']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='output', hue='exp', palette=exp_palette,
                kind='kde', col='construct', col_wrap=5,
                common_norm=False, levels=5, log_scale=True)

for construct, ax in g.axes_dict.items():
    for exp in data['exp'].unique():
        ax.axvline(gates.loc[gates['exp']==exp, 'marker'].values[0], c=exp_palette[exp], ls=':', zorder=0)
        ax.axhline(gates.loc[gates['exp']==exp, 'output'].values[0], c=exp_palette[exp], ls=':', zorder=0)

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT')]
display(df)

In [ ]:
quantiles, stats = base.calculate_bins_stats(df)

# Add metadata
stats = stats.merge(metadata, how='left', on='construct')

In [ ]:
fig, axes = plt.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.3)), figsize=(2,1.5))

biorep = 2
plot_df = quantiles[(quantiles['promoter']=='CMV') & (quantiles['design']<=1) & (quantiles['biorep']==biorep)]
line_kwargs = base.line_kwargs

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             style='construct', markers=main_markers, ax=ax, **line_kwargs)
ax.set(xscale='log', yscale='log', xlabel='marker', title='miR-FF5', ylim=(6e1,6e5))
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=base.font_sizes['smaller_size'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)

for ax in axes: ax.minorticks_off()

fig.savefig(rd.outfile(output_path/'miR-FF5-vary-ts-num_joint.pdf'), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.7), figsize=(2.4,1.5))
scatter_kwargs = base.scatter_kwargs
annotate_kwargs = base.annotate_kwargs

# stat gmean
plot_df = stats[(stats['promoter']=='CMV') & (stats['design']<=1)]
xlim = (-0.5, len(plot_df['construct'].unique())-0.5)
pairs = [('CL','base'), ('CL','OL')]

ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(4e3,1e5),)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e4,2e5))

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=(0.5,1))

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right',)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path/'miR-FF5-vary-ts-num_stats.pdf'), bbox_inches='tight')

# perform statistical tests
f, axes = plt.subplots(1,3, figsize=(5,2))
pairs = [('FF3x4','FF5x1'), ('FF3x4','FF5x4'), ('FF5x1','FF5x4')]
stat_list = ['output_gmean', 'output_std', 'slope']
for ax, stat in zip(axes, stat_list):
    sns.pointplot(data=plot_df, x='ts', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False,)
    if stat != 'slope': ax.set(yscale='log',)
    annotator = Annotator(ax, pairs, data=plot_df, x='ts', y=stat,)
    annotator.configure(**annotate_kwargs, line_offset=2).apply_and_annotate()

### Baselines

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'lenti')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata2.set_index('construct').to_dict('dict')
lenti_palette = metadata_dict['color']
lenti_markers = metadata_dict['markers']

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data2[(data2['construct'].isin(['UI','UT'])) | (data2['virus_dilution']==0)].groupby(['cell','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

In [ ]:
fig, axes = plt.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3), figsize=(2,1.2))
line_kwargs = base.line_kwargs

plot_df = quantiles2[(quantiles2['dox']==1000) & (quantiles2['moi']==7) & (quantiles2['cell']=='neuron') & (quantiles2['biorep']==1) &
                    ~(quantiles2['name'].str.contains('FXN')) & ~(quantiles2['name'].str.contains('FMRP'))].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=lenti_palette, 
             style='construct', markers=lenti_markers, ax=ax, hue_order=plot_order, **line_kwargs)
ax.set(xscale='log', yscale='log', xlabel='marker',)
baseline = baseline_df.loc[baseline_df['cell']=='neuron', 'output_gmean'].mean()
display(baseline)
ax.axhline(baseline, color='black', ls=':')

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=lenti_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/'neuron-joint-baseline.pdf'), bbox_inches='tight')
fig.savefig(rd.outfile(output_path/'neuron-joint-baseline.svg'), bbox_inches='tight')
fig.savefig(rd.outfile(output_path/'neuron-joint-baseline.png'), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.4), wspace=0.3), figsize=(2,1.2))
line_kwargs = base.line_kwargs

plot_df = quantiles2[(quantiles2['group'].isin(['controller','base'])) & (quantiles2['cell']=='iPS11') &
                    (quantiles2['biorep']==3) & (quantiles2['dox']==1000)].copy()
plot_df.sort_values(['ts_num','ts_kind'], ascending=False, inplace=True)
plot_order = reversed(plot_df['construct'].unique())

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=lenti_palette, 
             style='construct', markers=lenti_markers, ax=ax, hue_order=plot_order, **line_kwargs)
ax.set(xscale='log', yscale='log', xlabel='marker',)
ax.xaxis.set_minor_formatter('')
baseline = baseline_df.loc[baseline_df['cell']=='iPS11', 'output_gmean'].mean()
display(baseline)
ax.axhline(baseline, color='black', ls=':')

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=lenti_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(output_path/'iPS11-joint-baseline.pdf'), bbox_inches='tight')
fig.savefig(rd.outfile(output_path/'iPS11-joint-baseline.svg'), bbox_inches='tight')
fig.savefig(rd.outfile(output_path/'iPS11-joint-baseline.png'), bbox_inches='tight')

### CV

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'tuning')

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
# Alternate palette for promoter panel
metadata_alt_dict = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'promoters').set_index('construct').to_dict('dict')
promoter_palette = metadata_alt_dict['color']
promoter_markers = metadata_alt_dict['markers']

In [ ]:
# Plasmid titration
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'plasmid_titration')

In [ ]:
# Tuning: microRNA
pairs = [('base','CL'), ('OL','CL')]
miR_order = ['none','miR.FF4','miR.FF5','miRE.FF4','miRE.FF5']
stat = 'output_variation'
plot_df = stats[(((stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
annotate_kwargs = dict(test='t-test_ind', text_format='star', loc='inside', line_height=0, line_width=0.5, text_offset=-1)
f, axes = plt.subplots(1,len(miR_order)-1, figsize=(5,2), gridspec_kw=dict(wspace=0.6))
for ax, miR in zip(axes, miR_order[1:]):
    test_df = plot_df[(plot_df['miR']==miR) | (plot_df['group']=='base')]
    sns.pointplot(data=test_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False)
    ax.set(xlabel='', title=miR.replace('.','-'), ylabel='', xlim=(-0.5,2.5), ylim=(0,2.5))
    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, verbose=0).apply_and_annotate()
axes[0].set(ylabel='CV')

In [ ]:
# Tuning: promoter
pairs = [('base','CL'), ('OL','CL')]
promoter_order = ['EF1a','CAG','EFS','hPGK']
stat = 'output_variation'
annotate_kwargs = dict(test='t-test_ind', text_format='star', loc='inside', line_height=0, line_width=0.5, text_offset=-1)
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (stats['group']=='base'))].copy()
# remove outlier hPGK point
plot_df = plot_df[~((plot_df['promoter']=='hPGK') & (plot_df['exp']=='exp92') & (plot_df['biorep']==1))]

f, axes = plt.subplots(1,len(promoter_order), figsize=(5,2), gridspec_kw=dict(wspace=0.6))
for ax, promoter in zip(axes, promoter_order):
    test_df = plot_df[(plot_df['promoter']==promoter)]
    sns.pointplot(data=test_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=promoter_palette, legend=False)
    ax.set(xlabel='', title=promoter.replace('.','-'), ylabel='', xlim=(-0.5,2.5), ylim=(0,2.5))
    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, verbose=0).apply_and_annotate()
axes[0].set(ylabel='CV')

In [ ]:
# Tuning: # target sites
pairs = [('base','CL'), ('OL','CL')]
stat = 'output_variation'
ts_order = [1,2,4]
annotate_kwargs = dict(test='t-test_ind', text_format='star', loc='inside', line_height=0, line_width=0.5, text_offset=-1)
plot_df = stats[(((stats['group']=='controller') & (stats['miR']=='miRE.FF4') & (stats['design']==1)) | (stats['group']=='base')) & (stats['promoter']=='EF1a')].copy()

f, axes = plt.subplots(1,len(ts_order), figsize=(5,2), gridspec_kw=dict(wspace=0.6))
for ax, ts_num in zip(axes, ts_order):
    test_df = plot_df[(plot_df['ts_num'].isin([0,ts_num]))]
    sns.pointplot(data=test_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False)
    ax.set(xlabel='', title='TSx'+str(ts_num), ylabel='', xlim=(-0.5,2.5), ylim=(0,5))
    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, verbose=0).apply_and_annotate()
axes[0].set(ylabel='CV')

In [ ]:
# Plasmid titration
pairs = [('base','CL'), ('OL','CL')]
stat = 'output_variation'
annotate_kwargs = dict(test='t-test_ind', text_format='star', loc='inside', line_height=0, line_width=0.5, text_offset=-1)
plot_df = stats2[stats2['construct']!='AIO.039']
amt_order = sorted(plot_df['amount'].unique())

f, axes = plt.subplots(1,len(amt_order), figsize=(5,2), gridspec_kw=dict(wspace=0.6))
for ax, amt in zip(axes, amt_order):
    test_df = plot_df[(plot_df['amount']==amt)]
    sns.pointplot(data=test_df, x='ts_label', y=stat, ax=ax, hue='construct', palette=main_palette, legend=False)
    ax.set(xlabel='', title=str(amt), ylabel='', xlim=(-0.5,2.5), ylim=(0,2.5))
    annotator = Annotator(ax, pairs, data=test_df, x='ts_label', y=stat,)
    annotator.configure(**annotate_kwargs, verbose=0).apply_and_annotate()
axes[0].set(ylabel='CV')

In [ ]:
f, ax = plt.subplots(1,1, figsize=(2,2))
plot_df = stats2[stats2['construct']!='AIO.039']
line_kwargs2 = dict(legend=False, dashes=False, markersize=4, markeredgecolor='white', markeredgewidth=0.5)
sns.lineplot(data=plot_df, x='amount', y='output_variation', hue='construct', palette=main_palette, ax=ax,
             style='construct', markers=main_markers, **line_kwargs2)
ax.set(ylim=(0,2.5), xlabel='rel. plasmid amount', xticks=[0.5**i for i in range(5)]+[0], 
       xticklabels=['1.0','0.5','','','',''], xlim=(0,1.1), ylabel='CV')
sns.despine()

In [ ]:
# miRE-FF4 EF1a (best design) as a function of quantile bin
f, ax = plt.subplots(1,1, figsize=(2,2))
plot_df = quantiles[(((quantiles['miR']=='miRE.FF4') & (quantiles['group']=='controller') & (quantiles['design']==1) & 
                      (quantiles['ts'].isin(['FF4x1','FF6x1']))) | (quantiles['group']=='base')) & (quantiles['promoter']=='EF1a') &
                      (quantiles['exp']=='exp90.4')].copy()
line_kwargs2 = dict(legend=False, dashes=False, markersize=4, markeredgecolor='white', markeredgewidth=0.5)
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette,
             style='construct', markers=main_markers, **line_kwargs2, ax=ax,
             estimator=sp.stats.variation, errorbar='ci')
ax.set(xscale='log', ylabel='CV', xlabel='marker')